AsyncIO: coroutine을 IO에 특화시켜 편리하게 사용할 수 있다. <br>
Non-blocking 비동기 처리 <br>
- Blocking I/O: 호출된 함수가 자신의 작업이 완료될 때까지 제어권을 가지고 있어서 다른 함수는 대기해야 한다.
- Non-Blocking I/O: 호출된 함수(subroutine)가 return(yield) 후 호출한 함수(main routine)에 제어권 전달한다. 그래서 다른 함수는 일을 계속할 수 있다.

Thread 단점: 디버깅과 자원 접근할 때, race condition, deadlock을 고려해서 코딩해야 함. <br>
Coroutine의 장점: 하나의 루틴만 실행하기 때문에 lock 관리할 필요 없고 제어권으로 실행된다. <br>
Coroutine의 단점: 사용 함수가 비동기로 구현되어 있어야 한다. <br>

In [17]:
# 주피터 노트북은 자체 이벤트루프를 쓰기 대문에 새로 설치해줘야 한다.
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [18]:
import asyncio
import timeit
from urllib.request import urlopen # block 함수
from concurrent.futures import ThreadPoolExecutor
import threading
from bs4 import BeautifulSoup

start = timeit.default_timer()

# 서비스 방향이 비슷한 사이트로 실습 권장 (예: 게시판성 커뮤니티)
urls = ['https://daum.net', 'https://naver.com', 'https://mlbpark.donga.com',
        'https://tistory.com', 'https://wemakeprice.com']



In [19]:
async def fetch(url, executor):
    # thread 이름 출력
    print(f'thread Name : {threading.current_thread().getName()}, Start : {url}')

    # urlopen이 block함수라서 여기서 nonblock으로 만든다. 안 그러면 순차적으로 실행된다.
    res = await loop.run_in_executor(executor, urlopen, url)
    # (thread, function, target)


    soup = BeautifulSoup(res.read(), 'html.parser')
    # BeautifulSoup을 안 이용하고 res를 직접 반환하면 인간의 글씨로 표시되지 않는다.

    # 전체 페이지 소스 확인
    #print(soup.prettify())

    # title만
    result_data = soup.title

    # 완료된 것을 표시
    print(f'thread Name : {threading.current_thread().getName()}, Done : {url}')


    return result_data

# SyntaxError: 'await' outside async function
# await을 사용할 때, async를 안 쓰면 문법 오류이다.

In [20]:
async def main():

    # Thread pool 생성
    executor = ThreadPoolExecutor(max_workers = 10)

    # future 객체 모아서 gather에서 실행한다.
    futures = [
        asyncio.ensure_future(fetch(url, executor)) for url in urls
        
    ]
    # url 하나 당 하나의 thread

    #결과 취합
    rst = await asyncio.gather(*futures)
    print(f'Result : {rst}')

In [21]:
if __name__ == '__main__':
    # 루프 초기화
    loop = asyncio.get_event_loop()
    # 작업 완료 까지 대기
    loop.run_until_complete(main())
    # 수행 시간 계산
    duration = timeit.default_timer() - start
    # 총 실행 시간
    print('Total Running Time : ', duration)

thread Name : MainThread, Start : https://daum.net
thread Name : MainThread, Start : https://naver.com
thread Name : MainThread, Start : https://mlbpark.donga.com
thread Name : MainThread, Start : https://tistory.com
thread Name : MainThread, Start : https://wemakeprice.com
thread Name : MainThread, Done : https://mlbpark.donga.com
thread Name : MainThread, Done : https://daum.net
thread Name : MainThread, Done : https://tistory.com
thread Name : MainThread, Done : https://naver.com
thread Name : MainThread, Done : https://wemakeprice.com
Result : [<title>Daum</title>, <title>NAVER</title>, <title>↗ 파크에 오면 즐겁다 MLBPARK</title>, <title>TISTORY</title>, <title>특가프로 위메프로</title>]
Total Running Time :  2.03136590000031
